In [1]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [2]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.path_cost < node.path_cost

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [3]:
class State:
    def __init__(self,coordx,coordy,orientacion):
        self.coordx=coordx
        self.coordy=coordy
        self.orientacion=orientacion
        
    def modx(self,coordx):
        self.coordx=coordx
        
    def mody(self,coordy):
        self.coordy=coordy
        
    def modo(self,orientacion):
        self.orientacion=orientacion
        
    def getx(self):
        return self.coordx
        
    def gety(self):
        return self.coordy
        
    def geto(self):
        return self.orientacion
    
    def __repr__(self):
        return "Coordenadas ({},{}) Orientacion = {}".format(self.coordx,self.coordy,self.orientacion)
        
    def __eq__(self, other):
        return self.coordx == other.coordx and self.coordy == other.coordy and self.orientacion == other.orientacion
  
    def __hash__(self):
        return hash((self.coordx,self.coordy,self.orientacion))
    
    def __lt__(self,node):
        return ()


In [4]:
import numpy as np

def createboard(row,col):
  X = []
  for i in range(row):
    if i == 0 or i == row-1:     
      X.append([1]*col)
    else:
      X.append([0]*col)
  for i in range(row):
    X[i][0] = 1
    X[i][col-1] = 1
  return X

def toymaze():
  row = 11
  col = 11
  maze = createboard(row,col)
  
  for i in range(2,7):
    maze[i][2] = 1
    maze[i][8] = 1
    maze[6][i] = 1

  for i in range(2,5):
    maze[i][4] = 1
    maze[i][6] = 1

  for i in range(6,10):
    maze[i][4] = 1

  for i in range(2,9):
    maze[8][i] = 1
  
  maze[8][5] = 0
  maze[5][0] = 0

  maze[2][5] = maze[4][3] = maze[4][7] =  1
  maze[9][4] = maze[9][8] = 1
  
  printmaze(maze)
  
  return maze

def printmaze(maze):
  
  print("\n")  
  for row in maze:
    draw = ""
    for cell in row:
      if cell:
        draw = draw+"0"
      else:
        draw = draw+" "

    print(draw)
  print("\n")

laberinto = toymaze()
#laberinto = np.array(laberinto)
#print(laberinto,laberinto.shape)



00000000000
0         0
0 0 000 0 0
0 0 0 0 0 0
0 000 000 0
  0     0 0
0 00000 0 0
0   0     0
0 000 000 0
0   0   0 0
00000000000




In [5]:
class Laberinto(Problem):
    def __init__(self, initial, goal,maze):
        Problem.__init__(self,initial,goal)
        self.maze = maze
    
    def actions(self,state):
        return ["avanzar", "girari", "girard","girara"]

    def result(self,state,action):
        x=state.coordx
        y=state.coordy
        o=state.orientacion
        next_state=State(x,y,o)
        if (state.orientacion=='E'): #ESTE
            if (action=='avanzar'):
                if (self.maze[x][y+1]==1):
                    next_state.mody(y)
                else:
                    next_state.mody(y+1)
            elif (action=='girari'):
                next_state.modo('N')
            elif (action=='girard'):
                next_state.modo('S')
            elif (action=='girara'):
                next_state.modo('O')
            
        if (state.orientacion=='O'): #OESTE
            if (action=='avanzar'):
                if (self.maze[x][y-1]==1):
                    next_state.mody(y)
                else:
                    next_state.mody(y-1)
            elif (action=='girari'):
                next_state.modo('S')
            elif (action=='girard'):
                next_state.modo('N')
            elif (action=='girara'):
                next_state.modo('E')
        
        if (state.orientacion=='N'): #NORTE
            if (action=='avanzar'):
                if (self.maze[x-1][y]==1):
                    next_state.modx(x)
                else:
                    next_state.modx(x-1)
            elif (action=='girari'):
                next_state.modo('O')
            elif (action=='girard'):
                next_state.modo('E')
            elif (action=='girara'):
                next_state.modo('S')
                
        if (state.orientacion=='S'): #SUR
            if (action=='avanzar'):
                if (self.maze[x+1][y]==1):
                    next_state.modx(x)
                else:
                    next_state.modx(x+1)
            elif (action=='girari'):
                next_state.modo('E')
            elif (action=='girard'):
                next_state.modo('O')
            elif (action=='girara'):
                next_state.modo('N')
        return next_state
    
    def goal_test(self, state):
        if isinstance(self.goal, list):
            return state in self.goal  
        else:
            return state == self.goal  

    def path_cost(self, cost, state1, action, state2):
        return cost+1

# Pruebas

In [6]:
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

In [7]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [8]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        for child in node.expand(problem):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def g(n): return n.path_cost

def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


In [9]:
laberinto=toymaze()
goal=State(5,0,'O')
#coordenadas son(x=filas,y=columnas)
state0=State(5,5,'N')
statef=goal



00000000000
0         0
0 0 000 0 0
0 0 0 0 0 0
0 000 000 0
  0     0 0
0 00000 0 0
0   0     0
0 000 000 0
0   0   0 0
00000000000




In [10]:
problema=Laberinto(state0,statef,laberinto)

In [11]:
ucs=uniform_cost_search(problema)

In [12]:
print(ucs.solution())

['girard', 'avanzar', 'avanzar', 'girard', 'avanzar', 'avanzar', 'girari', 'avanzar', 'avanzar', 'girari', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'girari', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'girari', 'avanzar', 'avanzar', 'avanzar', 'avanzar', 'girard', 'avanzar']
